# Transform Data to Hand Coordiantes
This script shifts the data to the mean of the R-Markers and performs a change of basis to hand coordinates.

In [1]:
import pandas as pd
import numpy as np
import os
import pickle
from datetime import datetime

In [2]:
clean_data_path = 'CleanedData'
transformed_data_output_path = 'TransformedData'

In [3]:
def log(s):
    with open("transform_data_log.txt", "a") as myfile:
        myfile.write("[" + str(datetime.now()) + "] " + s + "\n")
    print("[" + str(datetime.now()) + "] " + s)

In [4]:
def unit_vector(vector):
    return vector / np.linalg.norm(vector)

In [5]:
def rotate(df):
    rs = df[df['label'].str.split('_').str[1] == "R"]
    
    c1 = rs[rs['label'].str.split('_').str[3] == "1"]
    c2 = rs[rs['label'].str.split('_').str[3] == "2"]
    c3 = rs[rs['label'].str.split('_').str[3] == "3"]
    c4 = rs[rs['label'].str.split('_').str[3] == "4"]
    
    m12 = np.array([(c1.XZero.values[0] + c2.XZero.values[0]) / 2, (c1.YZero.values[0] + c2.YZero.values[0]) / 2, (c1.ZZero.values[0] + c2.ZZero.values[0]) / 2])
    m14 = np.array([(c1.XZero.values[0] + c4.XZero.values[0]) / 2, (c1.YZero.values[0] + c4.YZero.values[0]) / 2, (c1.ZZero.values[0] + c4.ZZero.values[0]) / 2])
    m23 = np.array([(c3.XZero.values[0] + c2.XZero.values[0]) / 2, (c3.YZero.values[0] + c2.YZero.values[0]) / 2, (c3.ZZero.values[0] + c2.ZZero.values[0]) / 2])
    m34 = np.array([(c3.XZero.values[0] + c4.XZero.values[0]) / 2, (c3.YZero.values[0] + c4.YZero.values[0]) / 2, (c3.ZZero.values[0] + c4.ZZero.values[0]) / 2])

    # find three linear independent vectors vx, vy, vz
    vx = unit_vector(m23 - m14)
    vy = unit_vector(np.cross(vx, (m12 - m34)))
    vz = unit_vector(np.cross(vx, vy))

    baseOld = np.array([[1,0,0], [0,1,0], [0,0,1]]).T
    baseNew = np.array([vx, vy, vz]).T
    cob = np.linalg.solve(baseNew, baseOld)


    rotated = np.dot(cob, (np.array([df.XZero, df.YZero, df.ZZero])))
    df["XRot"] = rotated[0]
    df["YRot"] = rotated[1]
    df["ZRot"] = rotated[2]
    return df

In [6]:
for filename in os.listdir(clean_data_path):
    if not filename.endswith(".pkl"):
        continue
    log("Loading File: " + filename)
    errors = 0
    df = pd.read_pickle(clean_data_path + "/" + filename)
    
    # remove "hands:" from label column
    df.label = df.label.apply(lambda x: str(x).split(":")[1] if len(str(x).split(":")) > 1 else  str(x))


    dfOut = pd.DataFrame()
    
    # calculate mean of R_Shape, split left and right and shift to mean
    log("Shifting to R-Mean")
    # all right-hand data
    dfR = df[df['label'].str.split('_').str[0] == "R"].copy(deep=True)
    # rigid body data of right-hand data
    dfRR = dfR[dfR['label'].str.split('_').str[1] == "R"]
    dfMeanR = dfRR.groupby('time')[['x', 'y', 'z']].apply(np.mean)
    dfMeanR = dfMeanR.rename(columns={"x": "RXMean", "y": "RYMean", "z": "RZMean"})
    dfMeanR = dfMeanR.reset_index()
    dfR = pd.merge(dfR, dfMeanR, on="time")
    dfR["XZero"] = dfR["x"] - dfR["RXMean"]
    dfR["YZero"] = dfR["y"] - dfR["RYMean"]
    dfR["ZZero"] = dfR["z"] - dfR["RZMean"]

    # all left-hand data
    dfL = df[df['label'].str.split('_').str[0] == "L"].copy(deep=True)
    # rigid body data of left-hand data
    dfLR = dfL[dfL['label'].str.split('_').str[1] == "R"]
    dfMeanL = dfLR.groupby('time')[['x', 'y', 'z']].apply(np.mean)
    dfMeanL = dfMeanL.rename(columns={"x": "RXMean", "y": "RYMean", "z": "RZMean"})
    dfMeanL = dfMeanL.reset_index()
    dfL = pd.merge(dfL, dfMeanL, on="time")
    dfL["XZero"] = dfL["x"] - dfL["RXMean"]
    dfL["YZero"] = dfL["y"] - dfL["RYMean"]
    dfL["ZZero"] = dfL["z"] - dfL["RZMean"]
    
    for timestep in df['time'].unique():
        df_right_time_data = dfR[dfR.time == timestep].copy(deep=True)
        df_left_time_data = dfL[dfL.time == timestep].copy(deep=True)
        df_right_time_data.reset_index()
        df_left_time_data.reset_index()


        df_right_time_data = rotate(df_right_time_data)
        df_left_time_data = rotate(df_left_time_data)

        dfOut = dfOut.append(df_right_time_data)
        dfOut = dfOut.append(df_left_time_data)
    
    dfOut.to_pickle(transformed_data_output_path + "/" + filename.replace(".pkl","") + "_rotated.pkl") 
    log("Finished " + filename + " with " + str(errors) + " Errors")
log("Finished")

[2018-11-20 18:17:41.813485] Loading File: 07_1.pkl
[2018-11-20 18:17:42.793322] Shifting to R-Mean
[2018-11-20 18:46:09.652489] Finished 07_1.pkl with 0 Errors
[2018-11-20 18:46:09.653508] Loading File: 07_5.pkl
[2018-11-20 18:46:10.403242] Shifting to R-Mean
[2018-11-20 19:03:17.397429] Finished 07_5.pkl with 0 Errors
[2018-11-20 19:03:17.398711] Loading File: 07_0.pkl
[2018-11-20 19:03:18.424914] Shifting to R-Mean
[2018-11-20 19:37:30.154560] Finished 07_0.pkl with 0 Errors
[2018-11-20 19:37:30.155688] Loading File: 07_3.pkl
[2018-11-20 19:37:31.177808] Shifting to R-Mean
[2018-11-20 20:11:38.872237] Finished 07_3.pkl with 0 Errors
[2018-11-20 20:11:38.873906] Loading File: 07_4.pkl
[2018-11-20 20:11:39.896153] Shifting to R-Mean
[2018-11-20 20:44:45.910601] Finished 07_4.pkl with 0 Errors
[2018-11-20 20:44:45.911732] Loading File: 07_2.pkl
[2018-11-20 20:44:46.944247] Shifting to R-Mean
[2018-11-20 21:19:41.942522] Finished 07_2.pkl with 0 Errors
[2018-11-20 21:19:41.944155] Finis